In [35]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Load your dataset
data = pd.read_csv('Orders_New_query_2024_07_01.csv')

def recommend_items_with_similarity(area, item, data, n_recommendations=5):#colabborative
    # Step 1: Preprocess Dataset for Area-Item Matrix
    data['billing_address_state'] = data['billing_address_state'].str.strip().str.upper()
    
    # Create an area-item matrix
    area_item_matrix = data.pivot_table(index='billing_address_state', columns='sku_id', values='ordered_quantity', aggfunc='sum').fillna(0)

    # Compute item-item similarity using cosine similarity
    item_similarity_matrix = pd.DataFrame(cosine_similarity(area_item_matrix.T), index=area_item_matrix.columns, columns=area_item_matrix.columns)

    # Convert area to uppercase to match the format in the dataset
    area = area.strip().upper()
    
    # Check if the area is in the matrix
    if area not in area_item_matrix.index:
        return f"Area '{area}' not found in the dataset."

    # Get items bought in this area and sort by popularity
    items_in_area = area_item_matrix.loc[area].sort_values(ascending=False)

    # If the input item is not in the area or its quantity is zero, return a message
    if item not in items_in_area or items_in_area[item] == 0:
        return f"The item '{item}' has not been bought in area '{area}'. Popular items:\n{items_in_area.head(n_recommendations)}"
    
    # Recommend other items frequently bought in this area, excluding the given item
    recommendations = items_in_area[items_in_area.index != item].head(n_recommendations)
    
    # Get similarity scores for recommended items compared to the input item
    similarity_scores = item_similarity_matrix.loc[item, recommendations.index]
    
    # Combine recommendations with similarity scores
    recommendations_with_similarity = pd.DataFrame({
        'SKU ID': recommendations.index,
        'Ordered Quantity': recommendations.values,
        'Gross Merchandise Value': data.groupby('sku_id')['gross_merchandise_value'].sum().reindex(recommendations.index, fill_value=0).values,
    }).sort_values(by='Ordered Quantity', ascending=False)

    return recommendations_with_similarity[['SKU ID', 'Ordered Quantity', 'Gross Merchandise Value']]

def get_recommendations(sku_id, data, top_n=20):
    # Step 2: Preprocess Dataset for Product Profiles
    data.fillna(0, inplace=True)

    # Normalize numerical features
    scaler = MinMaxScaler()
    numerical_features = ['gross_merchandise_value', 'net_sales_before_tax', 'gift_wrap_expense',
                          'packaging_expense', 'handling_expense', 'shipping_expense']

    # Convert to numeric and coerce errors
    for col in numerical_features:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Drop rows with NaN values in numerical features
    data.dropna(subset=numerical_features, inplace=True)

    # Normalize numerical features
    data[numerical_features] = scaler.fit_transform(data[numerical_features])

    # One-hot encode categorical variables
    categorical_features = ['source', 'refund_status', 'rto_status', 'cancellation_status',
                            'order_status', 'payment_status', 'billing_address_state']
    data = pd.get_dummies(data, columns=categorical_features)

    # Create Product Profiles
    product_profiles = data.groupby('sku_id')[numerical_features].mean().reset_index()

    # Calculate Similarity Matrix
    feature_matrix = product_profiles.drop('sku_id', axis=1)
    similarity_matrix = cosine_similarity(feature_matrix)

    # Map SKU IDs to similarity matrix indices
    sku_id_to_index = {sku_id: index for index, sku_id in enumerate(product_profiles['sku_id'])}

    if sku_id not in sku_id_to_index:
        return pd.DataFrame(columns=['Recommended SKU', 'Ordered Quantity', 'Gross Merchandise Value'])

    index = sku_id_to_index[sku_id]
    similarity_scores = list(enumerate(similarity_matrix[index]))

    # Sort by similarity score in descending order
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get top N similar products excluding itself
    top_sku_indices = [i for i, score in similarity_scores[1:top_n + 1]]
    top_scores = [score for i, score in similarity_scores[1:top_n + 1]]

    # Get the recommended SKU IDs
    recommended_skus = product_profiles.iloc[top_sku_indices]['sku_id'].tolist()

    # Get ordered quantity and GMV for the recommended SKUs
    ordered_quantity = data.groupby('sku_id')['ordered_quantity'].sum().reindex(recommended_skus, fill_value=0).tolist()
    gmv = data.groupby('sku_id')['gross_merchandise_value'].sum().reindex(recommended_skus, fill_value=0).tolist()

    # Create a DataFrame to display recommendations along with similarity scores
    recommendation_df = pd.DataFrame({
        'SKU ID': recommended_skus,
        'Ordered Quantity': ordered_quantity,
        'Gross Merchandise Value': gmv,
        })
    return recommendation_df[['SKU ID', 'Ordered Quantity', 'Gross Merchandise Value']]

def analyze_orders(data):
    # Convert 'order_date_time_utc' to datetime format
    data['order_date_time_utc'] = pd.to_datetime(data['order_date_time_utc'], errors='coerce')

    # Filter the data for the specific months of February and March
    filtered_data = data[(data['order_date_time_utc'].dt.month.isin([2, 3])) & 
                         (data['order_date_time_utc'].dt.year == 2024)]

    # Exclude sku_id 0
    filtered_data = filtered_data[filtered_data['sku_id'] != 0]

    # Group by 'sku_id' and calculate total quantity, GMV, and number of cancellations
    sku_summary = filtered_data.groupby('sku_id').agg({
        'ordered_quantity': 'sum',
        'gross_merchandise_value': 'sum',
        'cancellation_status': lambda x: (x == 'TRUE').sum()  # Count of 'TRUE' status in cancellation_status
    }).reset_index()

    # Sort by 'ordered_quantity' and 'gross_merchandise_value' to find best and worst performers
    most_ordered = sku_summary.sort_values(by=['ordered_quantity', 'gross_merchandise_value'], ascending=False).head(10)  # Top 10 most ordered
    least_ordered = sku_summary.sort_values(by=['ordered_quantity', 'gross_merchandise_value'], ascending=True).head(10)  # Top 10 least ordered

    # Exclude 'cancellation_status' column for display
    most_ordered_display = most_ordered[['sku_id', 'ordered_quantity', 'gross_merchandise_value']]
    least_ordered_display = least_ordered[['sku_id', 'ordered_quantity', 'gross_merchandise_value']]

    return most_ordered_display, least_ordered_display


# Example usage
area = "TAMIL NADU"  # Replace with the desired area
input_item = 'amazon_B0CQKC4JBK'  # Replace with the SKU ID of the item input by the user

# Call the first function to get recommendations by area
recommendations_with_similarity = recommend_items_with_similarity(area, input_item, data, 20)
print(f"Recommended items for area '{area}' excluding item '{input_item}':\n", recommendations_with_similarity)

# Call the second function to get recommendations based on product similarity
recommendations_df = get_recommendations(input_item, data, top_n=10)
print(f"\nRecommended Products for SKU '{input_item}' with Similarity Scores:\n")
print(recommendations_df)

# Call the third function to analyze orders for February and March
most_ordered_display, least_ordered_display = analyze_orders(data)
print("\nMost Ordered Products (February and March 2024):")
print(most_ordered_display)

print("\nLeast Ordered Products (February and March 2024):")
print(least_ordered_display)


Recommended items for area 'TAMIL NADU' excluding item 'amazon_B0CQKC4JBK':
                                   SKU ID  Ordered Quantity  \
0                      amazon_B0CTMQPVJ2              67.0   
1                      amazon_B0CTMQ4YY7              53.0   
2   shopify_9315443441953_48922373062945              45.0   
3                      amazon_B0CTMQZ659              36.0   
4                      amazon_B0CTMQF324              36.0   
5                      amazon_B0CQK7NG2H              28.0   
6                      amazon_B0CQK7KQPW              27.0   
7                      amazon_B0CTQBR3S3              26.0   
8                      amazon_B0CTQH1B7H              23.0   
10  shopify_9315034366241_48921914212641              22.0   
9                      amazon_B0CQK69CQ5              22.0   
11  shopify_9313722171681_48920254611745              21.0   
12                     amazon_B0CQM6JMBP              18.0   
13                     amazon_B0CTQ1NHQQ              1